In [3]:
import io
import sys
import time
import warnings
from contextlib import redirect_stdout
from itertools import product
from typing import List, Tuple, cast  # noqa

import matplotlib.pyplot as plt
import networkx as nx  # noqa
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from IPython.display import Markdown, display
from pyomo.environ import *

import classiq
from classiq import *

warnings.filterwarnings("ignore")

In [4]:
from pyomo.environ import *

# Sample data: patient-donor pairs and compatibility scores
donors = ["donor1", "donor2", "donor3"]
patients = ["patient1", "patient2", "patient3"]
N = len(patients)
M = len(donors)
# Parameters
compatibility_scores = {
    ("donor1", "patient1"): 0.9,
    ("donor1", "patient2"): 0.7,
    ("donor1", "patient3"): 0.6,
    ("donor2", "patient1"): 0.8,
    ("donor2", "patient2"): 0.75,
    ("donor2", "patient3"): 0.65,
    ("donor3", "patient1"): 0.85,
    ("donor3", "patient2"): 0.8,
    ("donor3", "patient3"): 0.7,
}

# Create Pyomo model
model = ConcreteModel()

# Variables
model.x = Var(donors, patients, within=Binary)

# Objective
model.obj = Objective(
    expr=sum(
        compatibility_scores[donor, patient] * model.x[donor, patient]
        for donor in donors
        for patient in patients
    ),
    sense=maximize,
)

# Constraints
model.donor_constraint = ConstraintList()
for donor in donors:
    model.donor_constraint.add(
        sum(model.x[donor, patient] for patient in patients) <= 1
    )

model.patient_constraint = ConstraintList()
for patient in patients:
    model.patient_constraint.add(sum(model.x[donor, patient] for donor in donors) <= 1)

# Install "glpk" and unommente for runing this part
# Solve
# solver = SolverFactory("glpk")
# solver.solve(model)

# Output
print("\033[1m\033[4mOptimal solution:\033[0m")
for donor in donors:
    for patient in patients:
        if model.x[donor, patient].value == 1:
            print(f"{donor} donates kidney to {patient}")

print("\n\033[1m\033[4mModel Details\033[0m")
model.pprint()

Optimal solution:

Model Details
5 Set Declarations
    donor_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    patient_constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :    9 : {('donor1', 'patient1'), ('donor1', 'patient2'), ('donor1', 'patient3'), ('donor2', 'patient1'), ('donor2', 'patient2'), ('donor2', 'patient3'), ('donor3', 'patient1'), ('donor3', 'patient2'), ('donor3', 'patient3')}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'donor1', 'donor2', 'donor3'}
    x_index_1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dime

In [5]:
from classiq import (
    Preferences,
    construct_combinatorial_optimization_model,
    set_preferences,
)
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig

qaoa_config = QAOAConfig(num_layers=5, penalty_energy=2)

In [6]:
optimizer_config = OptimizerConfig(
    # opt_type='COBYLA',
    max_iteration=200,
    alpha_cvar=1,
)

In [7]:
qmod = construct_combinatorial_optimization_model(
    pyo_model=model,
    qaoa_config=qaoa_config,
    optimizer_config=optimizer_config,
)

# defining cosntraint such as computer and parameters for a quicker and more optimized circuit.
preferences = Preferences(transpilation_option="none", timeout_seconds=3000)

qmod = set_preferences(qmod, preferences)

In [8]:
from classiq import show, synthesize

qmod = set_constraints(qmod, Constraints(optimization_parameter="width"))
qprog = synthesize(qmod)
# show(qprog)

ClassiqAPIError: Call to API failed with code 401: Not authenticated
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack

In [ ]:
from classiq import execute

res = execute(qprog).result()

In [ ]:
from classiq.execution import VQESolverResult

vqe_result = res[0].value
vqe_result.convergence_graph

In [ ]:
import pandas as pd

from classiq.applications.combinatorial_optimization import (
    get_optimization_solution_from_pyo,
)

solution = get_optimization_solution_from_pyo(
    model, vqe_result=vqe_result, penalty_energy=qaoa_config.penalty_energy
)

optimization_result = pd.DataFrame.from_records(solution)

print("\n\033[1m\033[4mTop 10 Solutions\033[0m")
optimization_result.sort_values(by="cost", ascending=False).head(10)

In [ ]:
import matplotlib.pyplot as plt

optimization_result["cost"].plot(
    kind="hist", bins=30, edgecolor="black", weights=optimization_result["probability"]
)
plt.ylabel("Probability", fontsize=12)
plt.xlabel("Cost", fontsize=12)
plt.tick_params(axis="both", labelsize=12)
plt.title("Histogram of Cost Weighted by Probability", fontsize=16)
plt.show()

In [ ]:
# This function plots the solution in a table and a graph


def plotting_sol(x_sol, cost):
    x_sol_to_mat = np.reshape(np.array(x_sol), [N, M])  # vector to matrix
    print("\033[1m\033[4m** QAOA SOLUTION **\033[0m")
    print("\033[4mHighest Compatibility Score\033[0m = ", cost)

    # plotting in a table
    df = pd.DataFrame(x_sol_to_mat)
    df.columns = patients
    df.index = donors
    print(df)

    # plotting in a graph
    graph_sol = nx.DiGraph()
    graph_sol.add_nodes_from(donors + patients)
    for n, m in product(range(N), range(M)):
        if x_sol_to_mat[n, m] > 0:
            graph_sol.add_edges_from(
                [(donors[m], patients[n])],
                weight=compatibility_scores[(donors[m], patients[n])],
            )

    plt.figure(figsize=(10, 6))
    left = nx.bipartite.sets(graph_sol, top_nodes=patients)[0]
    pos = nx.bipartite_layout(graph_sol, left)

    nx.draw_networkx(
        graph_sol, pos=pos, nodelist=patients, font_size=22, font_color="None"
    )
    nx.draw_networkx_nodes(
        graph_sol, pos, nodelist=patients, node_color="#119DA4", node_size=500
    )
    for d in donors:
        x, y = pos[d]
        plt.text(
            x,
            y,
            s=d,
            bbox=dict(facecolor="#F43764", alpha=1),
            horizontalalignment="center",
            fontsize=12,
        )

    nx.draw_networkx_edges(graph_sol, pos, width=2)
    labels = nx.get_edge_attributes(graph_sol, "weight")
    nx.draw_networkx_edge_labels(
        graph_sol, pos, edge_labels=labels, font_size=12, label_pos=0.6
    )
    nx.draw_networkx_labels(
        graph_sol,
        pos,
        labels={co: co for co in patients},
        font_size=12,
        # font_color="#F4F9E9",
    )
    plt.title("Network Graph of the Best Solution", fontsize=16)
    plt.axis("off")
    plt.show()


# best_solution = optimization_result.loc[optimization_result.probability.idxmax()]
# plotting_sol(best_solution.solution, best_solution.probability)

best_solution = optimization_result.loc[optimization_result.cost.idxmax()]
plotting_sol(best_solution.solution, best_solution.cost)